### 连接mongo

In [197]:
from mongo_cli import db,MODE
sid_info = db["sid_info"]
res = sid_info.find({'status':0}).sort("_id",-1).limit(3)
print sid_info.find({'status':0}).sort("_id",-1).limit(3).count()
sid_list = [x.get('sid') for x in res]
print sid_list

C:\Users\Yulore\AppData\Local\conda\conda\envs\py2\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


1271449
[u'SID1d84aaf5f41a2ba7197fd7cb92b29781', u'SIDa6b453685b1e6b3efc41944e3a87abc5', u'SID114464cac6ec4e019bd25f9dee71266e']


### 获取计算时间

In [69]:
import datetime,time
from datetime import datetime as DT
def get_days(term, days=7, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        return [time.mktime((DT.strptime(term, frmt) - datetime.timedelta(days=i)).timetuple()) for i in xrange(days,-1, -1)]
get_days('20180930')

def get_days_between(start, end, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        days = (DT.strptime(end, frmt) - DT.strptime(start, frmt)).days
        return [time.mktime((DT.strptime(end, frmt) - datetime.timedelta(days=i)).timetuple()) for i in xrange(days,-1, -1)]
get_days_between('20181104','20181105')

[1541260800.0, 1541347200.0]

### 计算统计结果

In [176]:
import time

tim_list = get_days_between('20181015','20181107')
import pandas as pd
pct_a = []
pct_y = []
pct_m = []
pct_x = []
crawl_y = []
crawl_m = []
crawl_x = []
crawl_o = []
def cul(x,y):
    print x,y
    a = list(sid_info.find({'login_status':"0", 'end_time':{'$gt':x, '$lt':y}},
                           {'end_time':1,'start_time':1,'login_time':1,"crawler_channel":1,"_id":0,"status":1,"sid":1}))
    #                  .limit(3000))
    df = pd.DataFrame(a)
#     df['login_used'] = df.login_time-df.start_time
    df['crawl_used'] = df.end_time -df.login_time
#     df['report_used'] = df.report_used.map(float)
    df['all_used'] = df.end_time-df.start_time
    df['crawler_channel'][df.crawler_channel.isnull()]='yulore'
    df_ = df[df.crawl_used>=1200]
    pct_a.append(df_.shape[0]*100.0/df.shape[0])
    print df.crawler_channel.value_counts()
    crawl_y.append("{:.2f}".format(df[df.crawler_channel == 'yulore'].shape[0]*100.0/df.shape[0]))
    crawl_m.append("{:.2f}".format(df[df.crawler_channel == 'moxie'].shape[0]*100.0/df.shape[0]))
    crawl_x.append("{:.2f}".format(df[df.crawler_channel == 'xinde'].shape[0]*100.0/df.shape[0]))
    crawl_o.append("{:.2f}".format(df[df.crawler_channel == 'other'].shape[0]*100.0/df.shape[0]))
    pct_y.append(df_[df_.crawler_channel == 'yulore'].shape[0]*100.0/df.shape[0])
    pct_m.append(df_[df_.crawler_channel == 'moxie'].shape[0]*100.0/df.shape[0])
    pct_x.append(df_[df_.crawler_channel == 'xinde'].shape[0]*100.0/df.shape[0])
    return y
reduce(cul, tim_list)
print pct_a
print pct_y
print pct_m
print pct_x
print crawl_y
print crawl_m
print crawl_x
print crawl_o
# cul(1538236800.0, 1538323200.0)

1539532800.0 1539619200.0


C:\Users\Yulore\AppData\Local\conda\conda\envs\py2\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


yulore    5139
xinde      114
moxie       79
other        5
Name: crawler_channel, dtype: int64
1539619200.0 1539705600.0
yulore    6126
xinde      154
moxie      106
Name: crawler_channel, dtype: int64
1539705600.0 1539792000.0
yulore    5405
moxie     1432
xinde      261
Name: crawler_channel, dtype: int64
1539792000.0 1539878400.0
yulore    4171
moxie     1826
xinde      470
Name: crawler_channel, dtype: int64
1539878400.0 1539964800.0
yulore    5089
moxie     2187
xinde      480
Name: crawler_channel, dtype: int64
1539964800.0 1540051200.0
yulore    5658
moxie     2019
xinde      254
Name: crawler_channel, dtype: int64
1540051200.0 1540137600.0
yulore    5517
moxie     1993
xinde      243
Name: crawler_channel, dtype: int64
1540137600.0 1540224000.0
yulore    5615
moxie     2842
xinde      353
Name: crawler_channel, dtype: int64
1540224000.0 1540310400.0
yulore    3854
moxie     3054
xinde      207
other        9
Name: crawler_channel, dtype: int64
1540310400.0 1540396800.0
moxie  

### 单次计算

In [ ]:
x, y = get_days_between('20181101','20181102')
a = list(sid_info.find({'end_time':{'$gt':x, '$lt':y}},
                           {'end_time':1,'start_time':1,'login_time':1, 'crawler_channel':1,"_id":0,"status":1,"sid":1}))
#                  .limit(3000))
df = pd.DataFrame(a)
df['login_used'] = df.login_time-df.start_time
df['crawl_used'] = df.end_time -df.login_time
#     df['report_used'] = df.report_used.map(float)
df['all_used'] = df.end_time-df.start_time
df_timeout = df[df.crawl_used>=1200]
df_timeout['crawler_channel'][df_timeout.crawler_channel.isnull()]='yulore'

In [145]:
print df_timeout[df_timeout.crawler_channel == 'yulore'].shape[0]*100.0/sum(a.values())
a =  df_timeout['crawler_channel'].value_counts().map(int).to_dict()
print a.get('moxie',0)*100.0/sum(a.values())
print a.get('xinde',0)*100.0/sum(a.values())
print a.get('yulore',0)*100.0/sum(a.values())
# print df_timeout[(df_timeout.crawler_channel== "moxie")].shape[0]*1.0/df_timeout.shape[0]

4.90797546012
95.0920245399
0.0
4.90797546012


In [182]:
a = ['96.29', '95.93', '76.15', '64.50', '65.61', '71.34', '71.16', '63.73', '54.10', '47.68', '40.32', '23.83', '25.79', '30.43', '31.36', '32.13', '31.59', '20.11', '31.43', '33.83', '31.68', '33.26', '29.66']
b = ['1.48', '1.66', '20.17', '28.24', '28.20', '25.46', '25.71', '32.26', '42.87', '51.95', '59.44', '76.17', '74.19', '69.57', '68.61', '67.86', '68.41', '79.89', '59.23', '52.22', '57.74', '55.00', '50.96']
c = ['2.14', '2.41', '3.68', '7.27', '6.19', '3.20', '3.13', '4.01', '2.91', '0.37', '0.21', '0.00', '0.02', '0.00', '0.02', '0.01', '0.00', '0.00', '0.03', '0.05', '0.01', '0.00', '0.00']
d = ['0.09', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.13', '0.00', '0.03', '0.00', '0.00', '0.00', '0.02', '0.00', '0.00', '0.00', '9.31', '13.90', '10.57', '11.74', '19.39']
a = ['{:.1f}'.format(float(i)) for i in a]
b = ['{:.1f}'.format(float(i)) for i in b]
c = ['{:.1f}'.format(float(i)) for i in c]
d = ['{:.1f}'.format(float(i)) for i in d]
print a
print b
print c
print d


['96.3', '95.9', '76.2', '64.5', '65.6', '71.3', '71.2', '63.7', '54.1', '47.7', '40.3', '23.8', '25.8', '30.4', '31.4', '32.1', '31.6', '20.1', '31.4', '33.8', '31.7', '33.3', '29.7']
['1.5', '1.7', '20.2', '28.2', '28.2', '25.5', '25.7', '32.3', '42.9', '52.0', '59.4', '76.2', '74.2', '69.6', '68.6', '67.9', '68.4', '79.9', '59.2', '52.2', '57.7', '55.0', '51.0']
['2.1', '2.4', '3.7', '7.3', '6.2', '3.2', '3.1', '4.0', '2.9', '0.4', '0.2', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.1', '0.0', '0.0', '0.0']
['0.1', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.1', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '9.3', '13.9', '10.6', '11.7', '19.4']


In [ ]:
# 耗时
def used_stats(df, pattern='crawl_used'):
    used_min = df[pattern].min()
    used_max = df[pattern].max()
    used_mean = df[pattern].mean()
    used_75 = df[pattern].quantile(0.75)
    used_98 = df[pattern].quantile(0.98)
    return {
    "used_min": used_min,
    "used_max": used_max,
    "used_mean": used_mean,
    "used_75": used_75,
    "used_98": used_98,
    }
print used_stats(df,pattern='report_used')

In [6]:
df.sort_values('all_used')[df.all_used>=1200].shape[0]*1.0*100/df.shape[0]

C:\Users\Yulore\AppData\Local\conda\conda\envs\py2\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0.5999739141776445

In [40]:
a = [0.0, 0.24158757549611734, 0.0, 0.007896399241945672, 0.007237461098646595, 0.0, 0.0, 0.016002560409665547, 0.01335113484646195, 0.0, 0.0143000143000143, 0.05090785677922959, 0.0, 0.0, 0.01873711823121604, 0.0, 0.0, 0.0, 0.012893243940175348, 0.0, 0.012898232942086934, 0.011350737797956867, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018828845791752966, 0.03023888720895071, 4.25896323361498, 0.4350643339813015, 5.707282913165266, 0.5999739141776445, 0.9010919113749603]
a

[0.0,
 0.24158757549611734,
 0.0,
 0.007896399241945672,
 0.007237461098646595,
 0.0,
 0.0,
 0.016002560409665547,
 0.01335113484646195,
 0.0,
 0.0143000143000143,
 0.05090785677922959,
 0.0,
 0.0,
 0.01873711823121604,
 0.0,
 0.0,
 0.0,
 0.012893243940175348,
 0.0,
 0.012898232942086934,
 0.011350737797956867,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.018828845791752966,
 0.03023888720895071,
 4.25896323361498,
 0.4350643339813015,
 5.707282913165266,
 0.5999739141776445,
 0.9010919113749603]

### 检查&安装包

In [185]:
# !pip install pyecharts

In [186]:
import datetime,time
from datetime import datetime as DT
def get_days(term, days=34, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        return [(DT.strptime(term, frmt) - datetime.timedelta(days=i)).strftime(frmt) for i in range(days,-1, -1)]

def get_days_between(start, end, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        days = (DT.strptime(end, frmt) - DT.strptime(start, frmt)).days
        return [(DT.strptime(end, frmt) - datetime.timedelta(days=i)).strftime("%m%d") for i in range(days,-1, -1)]

b = get_days_between('20180701','20181106')

### 绘图

In [192]:
from pyecharts import Line, Bar
def get_days_between(start, end, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        days = (DT.strptime(end, frmt) - DT.strptime(start, frmt)).days
        return [(DT.strptime(end, frmt) - datetime.timedelta(days=i)).strftime("%m%d") for i in range(days,-1, -1)]

b = get_days_between('20180801','20181107')

a_7 = [0.14340344168260039, 0.26375282592313487, 0.1628332994097293, 0.03732736095558044, 0.0, 0.2823861630780092, 0.01966568338249754, 0.07123775601068566, 0.0, 0.03602954422626554, 0.0, 0.0, 0.027122321670735014, 0.0, 0.012484394506866416, 0.01279263144428809, 0.0136986301369863, 0.016603021749958494, 0.0, 0.5763302506366439, 0.4605263157894737, 0.3401360544217687, 0.5423602019824201, 0.06717420510523958, 0.09735202492211838, 0.07597340930674264, 0.0, 0.0, 0.019425019425019424, 0.013466199838405602, 0.0]
y_7 = [0.12746972594008923, 0.26375282592313487, 0.1628332994097293, 0.03732736095558044, 0.0, 0.2647370278856336, 0.01966568338249754, 0.07123775601068566, 0.0, 0.01801477211313277, 0.0, 0.0, 0.027122321670735014, 0.0, 0.012484394506866416, 0.0, 0.0, 0.016603021749958494, 0.0, 0.5763302506366439, 0.4605263157894737, 0.32123960695389264, 0.5423602019824201, 0.06717420510523958, 0.0778816199376947, 0.03798670465337132, 0.0, 0.0, 0.019425019425019424, 0.013466199838405602, 0.0]
m_7 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x_7 = [0.015933715742511154, 0.0, 0.0, 0.0, 0.0, 0.017649135192375574, 0.0, 0.0, 0.0, 0.01801477211313277, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01279263144428809, 0.0136986301369863, 0.0, 0.0, 0.0, 0.0, 0.01889644746787604, 0.0, 0.0, 0.019470404984423675, 0.03798670465337132, 0.0, 0.0, 0.0, 0.0, 0.0]

a_8 = [0.0, 0.020496003279360523, 0.0, 0.04640371229698376, 0.0, 1.0754955714888232, 0.0, 0.0, 0.0, 0.0, 0.016515276630883566, 0.016051364365971106, 0.034482758620689655, 0.01984126984126984, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03683241252302026, 0.0, 0.017828489926903193, 0.0, 0.02437538086532602, 0.025131942699170646, 0.16877637130801687, 0.02841716396703609, 0.118514616802739, 0.018597731076808628, 0.0, 0.22433132010353754, 0.0, 0.007896399241945672, 0.007237461098646595, 0.0, 0.0, 0.016002560409665547, 0.01335113484646195, 0.0, 0.0143000143000143, 0.05090785677922959, 0.0, 0.0, 0.01873711823121604, 0.0, 0.0, 0.0, 0.012893243940175348, 0.0, 0.012898232942086934, 0.011350737797956867, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018828845791752966, 0.03023888720895071, 4.25896323361498, 0.4350643339813015, 5.707282913165266, 0.5999739141776445, 0.848086504823492, 0.28303940779446984, 0.34825361057052134, 0.3917267314321529]
y_8 = [0.0, 0.020496003279360523, 0.0, 0.04640371229698376, 0.0, 1.0754955714888232, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01218769043266301, 0.012565971349585323, 0.0, 0.0, 0.05267316302343956, 0.0, 0.0, 0.22433132010353754, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01335113484646195, 0.0, 0.0, 0.05090785677922959, 0.0, 0.0, 0.01873711823121604, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018828845791752966, 0.0, 0.0, 0.0, 0.2801120448179272, 0.0, 0.11661189441323015, 0.0, 0.0, 0.0]
m_8 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0143000143000143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03023888720895071, 4.25896323361498, 0.4350643339813015, 5.427170868347339, 0.5999739141776445, 0.7314746104102618, 0.28303940779446984, 0.34825361057052134, 0.3917267314321529]
x_8 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016515276630883566, 0.016051364365971106, 0.034482758620689655, 0.01984126984126984, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03683241252302026, 0.0, 0.017828489926903193, 0.0, 0.01218769043266301, 0.012565971349585323, 0.16877637130801687, 0.02841716396703609, 0.06584145377929945, 0.018597731076808628, 0.0, 0.0, 0.0, 0.007896399241945672, 0.007237461098646595, 0.0, 0.0, 0.016002560409665547, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012893243940175348, 0.0, 0.012898232942086934, 0.011350737797956867, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
a = a_7 + a_8
y = y_7 + y_8
m = m_7 + m_8
x = x_7 + x_8
bar = Bar("20min超时占比（%）")
bar.add('all', b[:-1], a, is_datazoom_show=True, 
        mark_line=["average", "max"], datazoom_type='both',
        )
bar.add('yulore', b[:-1], y, is_datazoom_show=True, is_stack = True,
        mark_line=["average", "max"], datazoom_type='both',
        )
bar.add('moxie', b[:-1], m, is_datazoom_show=True,  is_stack = True,
        mark_line=["average", "max"], datazoom_type='both',
        )
bar.add('xinde', b[:-1], x, is_datazoom_show=True, is_stack = True,
        mark_line=["average", "max"], datazoom_type='both',
        )
bar

In [188]:
from pyecharts import Line, Bar
def get_days_between(start, end, frmt="%Y%m%d"):
        """
        计算周期获取，默认7天
        """
        days = (DT.strptime(end, frmt) - DT.strptime(start, frmt)).days
        return [(DT.strptime(end, frmt) - datetime.timedelta(days=i)).strftime("%m%d") for i in range(days,-1, -1)]

b = get_days_between('20181015','20181107')

c_y = ['96.3', '95.9', '76.2', '64.5', '65.6', '71.3', '71.2', '63.7', '54.1', '47.7', '40.3', '23.8', '25.8', '30.4', '31.4', '32.1', '31.6', '20.1', '31.4', '33.8', '31.7', '33.3', '29.7']
c_m = ['1.5', '1.7', '20.2', '28.2', '28.2', '25.5', '25.7', '32.3', '42.9', '52.0', '59.4', '76.2', '74.2', '69.6', '68.6', '67.9', '68.4', '79.9', '59.2', '52.2', '57.7', '55.0', '51.0']
c_x = ['2.1', '2.4', '3.7', '7.3', '6.2', '3.2', '3.1', '4.0', '2.9', '0.4', '0.2', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.1', '0.0', '0.0', '0.0']
c_o = ['0.1', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.1', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '9.3', '13.9', '10.6', '11.7', '19.4']
bar = Bar("爬虫使用占比（%）")
bar.add('yulore', b[:-1], c_y, is_datazoom_show=True, is_stack = True,
        #mark_line=["average", "max"], 
        is_label_show=True,label_pos='inside',
        datazoom_type='both',
        )
bar.add('moxie', b[:-1], c_m, is_datazoom_show=True,  is_stack = True,
        #mark_line=["average", "max"], 
        is_label_show=True,label_pos='inside',
        datazoom_type='both',
        )
bar.add('xinde', b[:-1], c_x, is_datazoom_show=True, is_stack = True,
        #mark_line=["average", "max"],
        is_label_show=True,label_pos='inside',
        datazoom_type='both',
        )
bar.add('other', b[:-1], c_o, is_datazoom_show=True, is_stack = True,
        #mark_line=["average", "max"],
        is_label_show=True,label_pos='outside',
        datazoom_type='both',
        )
bar

In [196]:
from pyecharts import Funnel
# fl = Funnel("超时漏斗图", "", title_pos='left', width=1000, height=3000
#             )
# fl.add("按日期排", b[1:], a, is_label_show=False, 
#         label_pos='inside', label_text_color='white',
#         xaxis_margin = 1
#         )
# fl